In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys

# Project root (local repository path)
repo_root = "/Users/jakobfaber/Documents/research/caltech/ovro/dsa110/chime_dsa_codetections/FLITS"
# Ensure module import path for scat_analysis
sys.path.insert(0, str(Path(repo_root) / "scattering"))

# Axes
time = np.linspace(-10.0, 10.0, 512)       # ms
freq = np.linspace(0.4, 0.8, 128)          # GHz
df_MHz = (freq[1]-freq[0]) * 1000.0

# Import model
from scat_analysis.burstfit import FRBModel, FRBParams

# Build model and truth
m = FRBModel(time=time, freq=freq, data=None, dm_init=0.0, df_MHz=df_MHz)
p_true = FRBParams(c0=1.0, t0=0.0, gamma=-1.6, zeta=0.2, tau_1ghz=0.5, alpha=4.4, delta_dm=0.0)

model = m(p_true, "M3")

# Add noise
rng = np.random.default_rng(0)
noise_std = 0.1
data = model + rng.normal(0, noise_std, size=model.shape)

# Save locally (relative to notebook dir)
out_path = Path("synthetic_dynspec.npy")
np.save(out_path, data.astype(np.float64))
print("Saved synthetic spectrum:", out_path.resolve())


In [ ]:
plt.figure(figsize=(6,4))
extent = [time[0], time[-1], freq[0], freq[-1]]
plt.imshow(data, aspect='auto', origin='lower', extent=extent, cmap='plasma')
plt.xlabel("Time (ms)")
plt.ylabel("Frequency (GHz)")
plt.title("Synthetic dynamic spectrum")
plt.colorbar(label="I (arb)")
plt.show()


In [ ]:
# Gaussian likelihood, alpha fixed
from scat_analysis.burstfit_pipeline import BurstPipeline

inpath = Path("synthetic_dynspec.npy")
outdir = Path(".")

pipe = BurstPipeline(
    name="syn",
    inpath=inpath,
    outpath=outdir,
    dm_init=0.0,
    steps=600,
    nproc=0,
    yes=True,
    likelihood="gaussian",
    alpha_fixed=4.4,
    delta_dm_sigma=0.1,
    sample_log_params=True,
    f_factor=1, t_factor=1,
    telescope="chime",
    telcfg_path=str(Path(repo_root)/"scattering/configs/telescopes.yaml"),
    sampcfg_path=str(Path(repo_root)/"scattering/configs/sampler.yaml"),
)
res = pipe.run_full(model_scan=False, plot=True, save=True, show=False)
print("best_key:", res["best_key"])
if res["best_key"] == "M3":
    print("best_params:", res["best_params"])
print("GoF reduced chi2:", res["goodness_of_fit"]["chi2_reduced"])


In [ ]:
# Student-t likelihood, alpha free with Gaussian prior
pipe_t = BurstPipeline(
    name="syn_t",
    inpath=inpath,
    outpath=outdir,
    dm_init=0.0,
    steps=600,
    nproc=0,
    yes=True,
    likelihood="studentt",
    studentt_nu=5.0,
    alpha_mu=4.4, alpha_sigma=0.6,
    delta_dm_sigma=0.1,
    sample_log_params=True,
    f_factor=1, t_factor=1,
    telescope="chime",
    telcfg_path=str(Path(repo_root)/"scattering/configs/telescopes.yaml"),
    sampcfg_path=str(Path(repo_root)/"scattering/configs/sampler.yaml"),
)
res_t = pipe_t.run_full(model_scan=False, plot=True, save=True, show=False)
print("best_key:", res_t["best_key"])
if res_t["best_key"] == "M3":
    print("best_params:", res_t["best_params"])
print("GoF reduced chi2:", res_t["goodness_of_fit"]["chi2_reduced"])
